In [52]:
import numpy as np
from scipy.optimize import minimize, rosen
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
#from sklearn.gaussian_process.kernels import ConstantKernel, WhiteKernel, RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, Sum, Product
from sklearn.gaussian_process.kernels import ConstantKernel as C, WhiteKernel as W, RBF, Matern, RationalQuadratic as RQ, ExpSineSquared as ESS, DotProduct as DP, Sum, Product
import warnings
warnings.filterwarnings("ignore")

#===========================================================================================================#
#Obtaining initial values

def conductivity_func(X, finding_min=False):
    if finding_min==False:
        return (rosen(X.T) * 1e-8).round(6)
    elif finding_min==True:
        return -(rosen(X.T) * 1e-8).round(6)

np.random.seed(10003)
samples = 12
concentrations = 4
X = (np.random.random(samples*concentrations).reshape(samples, concentrations)*10).round(3)
y = conductivity_func(X)

for i in range(len(y)):
    print(f'Sample {i+1}: {X[i].round(3)}, {y[i]}')


#===========================================================================================================#
#Best Kernel

kernel = 31.6**2 * RBF(length_scale=10, length_scale_bounds=(10.0, 10000.0)) + DP(sigma_0=1e+04, sigma_0_bounds=(0.001, 10000.0))
model = GaussianProcessRegressor(kernel = kernel, normalize_y=True, alpha=1e-8)
'''
score = cross_val_score(model, X, -y, cv=11, scoring='neg_mean_absolute_error')
print(score)
print(np.mean(score))
   ''' 
#===========================================================================================================#
#Cross fold testing
'''
param_grid = [
    {'alpha': [10e-10, 10e-9, 10e3, 10e4, 10e5]}
]

param_search = GridSearchCV(model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=11, n_jobs=-1)

param_search.fit(X, -y)

best_parameters = param_search.best_params_
best_score = param_search.best_score_

print(best_parameters)
print(best_score)
'''
#===========================================================================================================#
#Baysian Optimization

def guess_in_bounds(bounds):
    return [(b[1] - b[0]) * np.random.random() + b[0] for b in bounds]

def acquisition_function1():
    return

bounds = [(0,10)]*concentrations

#===========================================================================================================#
#Global Minimum

model.fit(X, y)

def min_over_bounds(func, bounds, model = None):
    global_mins = []
    if model == None:
        for i in range(12):
            np.random.seed(i)
            guess = guess_in_bounds(bounds)
            global_min = minimize(func, guess, bounds=bounds, method='Nelder-Mead', args = True)
            global_mins.append((global_min.x, -global_min.fun))
    else:
        for i in range(12):
            np.random.seed(i)
            guess = guess_in_bounds(bounds)
            global_min = minimize(func, guess, bounds=bounds, method='Nelder-Mead', args = model)
            global_mins.append((global_min.x, -global_min.fun))
    print(global_mins)
    return max(global_mins, key=lambda x: x[1])

def ML_func(guess, model):
    return -model.predict(guess.reshape(1, -1))

rosen_min = min_over_bounds(conductivity_func, bounds)
GPR_min = min_over_bounds(ML_func, bounds, model)
print(rosen_min)
print(GPR_min)
print(conductivity_func(GPR_min[0]))
print(model.predict(np.array([[10., 10., 10.,  0.]])))
print(np.array([[10., 10., 10.,  0.]]))

Sample 1: [9.206 5.806 9.795 7.452], 0.014636
Sample 2: [9.123 5.265 8.88  5.74 ], 0.011781
Sample 3: [0.661 2.222 3.34  7.025], 2.3e-05
Sample 4: [2.022 2.641 2.202 2.139], 3.2e-05
Sample 5: [0.501 2.135 3.23  1.475], 8.6e-05
Sample 6: [0.363 1.749 2.184 1.032], 1.7e-05
Sample 7: [5.306 5.682 0.108 5.365], 0.001569
Sample 8: [0.608 8.135 1.662 2.234], 0.004223
Sample 9: [5.049 3.675 4.728 4.297], 0.000879
Sample 10: [7.569 8.265 5.136 5.122], 0.006847
Sample 11: [0.16  6.248 5.637 4.814], 0.001882
Sample 12: [3.649 6.6   2.837 8.771], 0.001704
[(array([10., 10., 10.,  0.]), 0.026202), (array([1.00000000e+01, 1.00000000e+01, 1.14893991e-03, 0.00000000e+00]), 0.018101), (array([10.,  0., 10.,  0.]), 0.020102), (array([10., 10.,  0.,  0.]), 0.018102), (array([10., 10., 10.,  0.]), 0.026202), (array([ 0.01952995, 10.        ,  0.        , 10.        ]), 0.010201), (array([ 9.99980544,  0.        , 10.        ,  0.        ]), 0.020101), (array([ 0.05827182, 10.        , 10.        ,  0.   

kernel = 31.6**2 * RBF(length_scale=10, length_scale_bounds=(10.0, 10000.0)) + DP(sigma_0=1e+04, sigma_0_bounds=(0.001, 10000.0))

model = GaussianProcessRegressor(kernel = kernel, normalize_y=True, alpha=1e-8)

score = -5.251450383038068e-06

error: -0.0012242895406499746